<a href="https://colab.research.google.com/github/orel006/128-356-Big-Data/blob/main/ClassLab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#นาย บุญญฤทธิ์ ก้องวิริยะกุล  รหัสนักศึกษา 6704800022
#นาย สหรัฐ สมดา รหัสนักศึกษา 6704800024
#นาย ชาคริต ทองวิเศษ รหัสนักศึกษา 6704800031
#นาย พิสิษฐ์ แซ่เตีย รหัสนักศึกษา 6704800038
##นาย อมรเทพ จันทร์เมฆา รหัสนักศึกษา 6704800051

In [ ]:
!curl -L -o ./brazilian-ecommerce.zip\
https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42.6M  100 42.6M    0     0  15.7M      0  0:00:02  0:00:02 --:--:-- 28.7M


In [ ]:
!unzip /content/brazilian-ecommerce.zip

Archive:  /content/brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

# สร้าง Spark Session
spark = SparkSession.builder \
    .appName("Olist_Ecommerce_Analysis") \
    .getOrCreate()

In [ ]:
customers_df = spark.read.csv('olist_customers_dataset.csv', header=True, inferSchema=True)
orders_df = spark.read.csv('olist_orders_dataset.csv', header=True, inferSchema=True)
order_items_df = spark.read.csv('olist_order_items_dataset.csv', header=True, inferSchema=True)
products_df = spark.read.csv('olist_products_dataset.csv', header=True, inferSchema=True)
payments_df = spark.read.csv('olist_order_payments_dataset.csv', header=True, inferSchema=True)
reviews_df = spark.read.csv('olist_order_reviews_dataset.csv', header=True, inferSchema=True)
sellers_df = spark.read.csv('olist_sellers_dataset.csv', header=True, inferSchema=True)
category_translation_df = spark.read.csv('product_category_name_translation.csv', header=True, inferSchema=True)

In [ ]:
# ตรวจสอบโครงสร้างก่อนจัดการ
print("Before Schema Update:")
orders_df.printSchema()

# รายชื่อคอลัมน์ที่ต้องเป็นวันที่
timestamp_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

# วนลูปเปลี่ยน Type เป็น Timestamp
for col_name in timestamp_cols:
    orders_df = orders_df.withColumn(col_name, to_timestamp(col(col_name)))

# ตรวจสอบอีกครั้ง จะเห็นว่าเปลี่ยนจาก string เป็น timestamp แล้ว
print("After Schema Update:")
orders_df.printSchema()

Before Schema Update:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)

After Schema Update:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [ ]:
# สร้าง View สำหรับแต่ละ DataFrame
customers_df.createOrReplaceTempView("customers")
orders_df.createOrReplaceTempView("orders")
order_items_df.createOrReplaceTempView("order_items")
products_df.createOrReplaceTempView("products")
payments_df.createOrReplaceTempView("payments")
reviews_df.createOrReplaceTempView("reviews")
sellers_df.createOrReplaceTempView("sellers")
category_translation_df.createOrReplaceTempView("category_translation")

print("✅ Task 1 สำเร็จ: โหลดข้อมูลและจัดการ Type เรียบร้อย พร้อมใช้ SQL แล้ว!")

✅ Task 1 สำเร็จ: โหลดข้อมูลและจัดการ Type เรียบร้อย พร้อมใช้ SQL แล้ว!


In [ ]:
# สร้าง df_master จากการ Join 4 ตารางหลัก
df_master = spark.sql("""
    SELECT
        o.order_id,
        o.customer_id,
        o.order_status,
        o.order_purchase_timestamp,
        i.product_id,
        i.seller_id,
        i.price,
        i.freight_value,
        p.product_category_name,
        c.customer_unique_id,
        c.customer_city,
        c.customer_state
    FROM orders o
    JOIN order_items i ON o.order_id = i.order_id
    JOIN products p ON i.product_id = p.product_id
    JOIN customers c ON o.customer_id = c.customer_id
""")

# แสดงโครงสร้างและข้อมูลตัวอย่าง
df_master.show(5)

+--------------------+--------------------+------------+------------------------+--------------------+--------------------+-----+-------------+---------------------+--------------------+---------------+--------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|          product_id|           seller_id|price|freight_value|product_category_name|  customer_unique_id|  customer_city|customer_state|
+--------------------+--------------------+------------+------------------------+--------------------+--------------------+-----+-------------+---------------------+--------------------+---------------+--------------+
|00018f77f2f0320c5...|f6dd3ec061db4e398...|   delivered|     2017-04-26 10:53:06|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|239.9|        19.93|             pet_shop|eb28e67c4c0b83846...|santa fe do sul|            SP|
|000229ec398224ef6...|6489ae5e4333f3693...|   delivered|     2018-01-14 14:33:31|c777355d18b72b67a...|5b51032eddd242adc...|199.0

In [ ]:
original_count = order_items_df.count()
master_count = df_master.count()

print(f"จำนวนรายการใน order_items: {original_count:,}")
print(f"จำนวนรายการใน df_master: {master_count:,}")

if original_count == master_count:
    print("✅ Join สำเร็จ: ข้อมูลครบถ้วนและไม่บวม")
else:
    print("⚠️ ข้อควรระวัง: จำนวนแถวไม่เท่ากัน อาจมีข้อมูลบางส่วนที่ไม่ Match กันในตารางหลัก")

จำนวนรายการใน order_items: 112,650
จำนวนรายการใน df_master: 112,650
✅ Join สำเร็จ: ข้อมูลครบถ้วนและไม่บวม


In [ ]:
# นับจำนวนแถวจากตารางต้นทาง และ ตารางที่ Join เสร็จแล้ว
count_orders = orders_df.count()
count_items = order_items_df.count()
count_master = df_master.count()

print(f"📦 จำนวนออเดอร์ทั้งหมด (Orders): {count_orders:,}")
print(f"🛒 จำนวนรายการสินค้าทั้งหมด (Items): {count_items:,}")
print(f"🚀 จำนวนแถวหลัง Join (df_master): {count_master:,}")

📦 จำนวนออเดอร์ทั้งหมด (Orders): 99,441
🛒 จำนวนรายการสินค้าทั้งหมด (Items): 112,650
🚀 จำนวนแถวหลัง Join (df_master): 112,650


In [ ]:
# เช็กว่ามี items ไหนที่หาข้อมูลสินค้า (products) ไม่เจอหรือไม่
check_nulls = df_master.filter(col("product_category_name").isNull()).count()

print(f"❌ รายการสินค้าที่ไม่พบหมวดหมู่: {check_nulls:,} แถว")

❌ รายการสินค้าที่ไม่พบหมวดหมู่: 1,603 แถว


In [ ]:
df_master.createOrReplaceTempView("master")

In [ ]:
# ใช้ SQL หา 5 อันดับหมวดสินค้าทำเงิน
top_products = spark.sql("""
    SELECT
        product_category_name,
        ROUND(SUM(price), 2) AS total_sales
    FROM master
    WHERE product_category_name IS NOT NULL
    GROUP BY product_category_name
    ORDER BY total_sales DESC
    LIMIT 5
""")

top_products.show()

+---------------------+-----------+
|product_category_name|total_sales|
+---------------------+-----------+
|         beleza_saude| 1258681.34|
|   relogios_presentes| 1205005.68|
|      cama_mesa_banho| 1036988.68|
|        esporte_lazer|  988048.97|
| informatica_acess...|  911954.32|
+---------------------+-----------+



In [ ]:
# ใช้ SQL หา 5 อันดับเมืองที่สั่งซื้อบ่อยที่สุด
top_cities = spark.sql("""
    SELECT
        customer_city,
        COUNT(DISTINCT order_id) AS total_orders
    FROM master
    GROUP BY customer_city
    ORDER BY total_orders DESC
    LIMIT 5
""")

top_cities.show()

+--------------+------------+
| customer_city|total_orders|
+--------------+------------+
|     sao paulo|       15402|
|rio de janeiro|        6834|
|belo horizonte|        2750|
|      brasilia|        2116|
|      curitiba|        1510|
+--------------+------------+



In [ ]:
# วิเคราะห์แนวโน้มรายเดือน (Trend Analysis)
monthly_sales = spark.sql("""
    SELECT
        DATE_FORMAT(order_purchase_timestamp, 'yyyy-MM') AS month,
        ROUND(SUM(price), 2) AS monthly_revenue
    FROM master
    WHERE order_purchase_timestamp IS NOT NULL
    GROUP BY month
    ORDER BY month ASC
""")

monthly_sales.show()

+-------+---------------+
|  month|monthly_revenue|
+-------+---------------+
|2016-09|         267.36|
|2016-10|       49507.66|
|2016-12|           10.9|
|2017-01|      120312.87|
|2017-02|      247303.02|
|2017-03|       374344.3|
|2017-04|      359927.23|
|2017-05|      506071.14|
|2017-06|       433038.6|
|2017-07|      498031.48|
|2017-08|      573971.68|
|2017-09|      624401.69|
|2017-10|      664219.43|
|2017-11|     1010271.37|
|2017-12|      743914.17|
|2018-01|      950030.36|
|2018-02|      844178.71|
|2018-03|      983213.44|
|2018-04|      996647.75|
|2018-05|      996517.68|
+-------+---------------+
only showing top 20 rows
